# TODO

- Word stemming
- Consider removing not *all* numbers; would date numbers/years proabably be important?

# Parsing + Preprocessing Data

In [1]:
from bs4 import BeautifulSoup
import string
import nltk
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
import ipdb
from random import choices

## Helper Functions

In [2]:
def tokenize_doc(doc):
    '''Convert to lowercase, remove punctuation/whitespace, and split on spaces'''
    doc = doc.lower()
    punc = string.punctuation.replace('\'', '')  # Don't remove apostrophes
    whitespace = string.whitespace + '\x03'  # End of file char
    
    # Convert whitespace/punctuation to spaces, and remove apostophes 
    translator = str.maketrans(punc + whitespace, ' ' * (len(punc) + len(whitespace)), '\'')
    doc_no_punc = doc.translate(translator)
    return doc_no_punc.split()


def remove_stop(tokens, stopwords):
    return [token for token in tokens if token not in stopwords]

## Parse sgm file for all articles

In [102]:
with open('./data/reuters21578/reut2-000.sgm') as f:
    data = f.read()

soup = BeautifulSoup(data, features="html5lib")
texts = soup.find_all('text')

## Find the title + article and tokenize

In [103]:
title_docs = {}
for text in texts:
    title = text.findChild('title')
    
    # Title is non-existent for a few articles
    if title:
        
        # Use contents because the text has no name; always the last element
        title_docs[title.string] = text.contents[-1]

# Example title: BAHIA COCOA REVIEW
title_doc_token = {title: tokenize_doc(doc) for title, doc in title_docs.items()}

## Remove stop words

In [104]:
#nltk.download()
stop_words = stopwords.words('English')
stop_words += ['said', 'reuter']

# Remove apostrophes; cleaned words have them removed
stop_words = set([word.replace('\'', '') for word in stop_words])
title_docs_token_no_stop = {title: remove_stop(tokens, stop_words) for title, tokens in title_doc_token.items()}

## Stem words

In [105]:
from nltk.stem import PorterStemmer

ps = PorterStemmer()

# Stem each word in each document
title_docs_stemmed = {title: [ps.stem(w) for w in doc] for title, doc in title_docs_token_no_stop.items()}

### Remove articles with "blah blah blah" as only text

In [106]:
title_docs_stemmed = {title: words for title, words in title_docs_stemmed.items() if len(words) != 3}

## Create list of unique words

In [107]:
unique_words = set().union(*title_docs_stemmed.values())
unique_words = list(unique_words)

# Try to convert to number; if it is, we don't want it
for num_word in unique_words.copy():
    try:
        int(num_word)
    except ValueError:
        continue

    for doc, words in title_docs_stemmed.items():
        title_docs_stemmed[doc] = [word for word in words if word != num_word]
    unique_words.remove(num_word)

# Starting Gibbs

## Initialize Counts

In [109]:
K = 20
titles = title_docs_stemmed.keys()
document_word_topics = {title: [] for title in titles}  # Contains the ordered list of topics for each document
                                                        # Dict of lists
    
# Counts of each topic per document (dict of dicts)
document_topic_counts = {title: dict.fromkeys(range(1, K + 1), 0) for title in titles}
word_topic_counts = {word: dict.fromkeys(range(1, K + 1), 0) for word in unique_words}  # Counts of each topic per word (dict of dicts)
total_topic_counts = dict.fromkeys(range(1, K + 1), 0)  # Counts of each topic across all documents

In [110]:
for doc, words in title_docs_stemmed.items():
    for i, word in enumerate(words):
        topic = np.random.randint(1, K + 1)
        document_word_topics[doc].append(topic)
        document_topic_counts[doc][topic] = document_topic_counts[doc].get(topic, 0) + 1
        word_topic_counts[word][topic] = word_topic_counts[word].get(topic, 0) + 1
        total_topic_counts[topic] = total_topic_counts[topic] + 1

Topics of the word in document

In [20]:
document_word_topics['BAHIA COCOA REVIEW']

[4,
 6,
 6,
 17,
 16,
 12,
 19,
 4,
 9,
 7,
 10,
 1,
 9,
 12,
 4,
 9,
 6,
 11,
 3,
 13,
 4,
 18,
 18,
 17,
 7,
 6,
 1,
 5,
 20,
 15,
 11,
 4,
 10,
 12,
 7,
 18,
 6,
 16,
 13,
 9,
 10,
 17,
 13,
 8,
 8,
 8,
 5,
 6,
 1,
 9,
 5,
 2,
 13,
 5,
 18,
 7,
 6,
 8,
 20,
 15,
 1,
 2,
 15,
 4,
 20,
 20,
 1,
 19,
 4,
 14,
 18,
 3,
 10,
 4,
 16,
 13,
 6,
 13,
 15,
 14,
 20,
 9,
 15,
 20,
 15,
 5,
 7,
 15,
 5,
 18,
 5,
 13,
 1,
 8,
 7,
 17,
 7,
 20,
 20,
 20,
 13,
 19,
 10,
 13,
 6,
 15,
 20,
 5,
 12,
 10,
 4,
 7,
 9,
 9,
 4,
 4,
 10,
 1,
 13,
 19,
 1,
 5,
 1,
 4,
 14,
 11,
 19,
 13,
 1,
 12,
 15,
 3,
 1,
 17,
 15,
 5,
 12,
 3,
 2,
 3,
 2,
 12,
 17,
 20,
 7,
 2,
 15,
 14,
 20,
 9,
 3,
 11,
 12,
 15,
 9,
 7,
 8,
 2,
 13,
 3,
 20,
 4,
 15,
 2,
 15,
 20,
 18,
 7,
 9,
 3,
 3,
 7,
 8,
 8,
 1,
 2,
 10,
 14,
 16,
 20,
 12,
 11,
 10,
 4,
 19,
 12,
 15,
 12,
 2,
 16,
 7,
 18,
 19,
 17,
 13,
 2,
 5,
 9,
 18,
 12,
 5,
 8,
 5,
 1,
 3,
 5,
 18,
 11,
 5,
 6,
 13,
 8,
 8,
 16,
 19,
 3,
 1,
 2,
 19,
 2,
 10,
 18,
 3

Topic distribution in document

In [21]:
document_topic_counts['BAHIA COCOA REVIEW']

{1: 17,
 2: 16,
 3: 15,
 4: 18,
 5: 17,
 6: 14,
 7: 14,
 8: 12,
 9: 12,
 10: 13,
 11: 8,
 12: 16,
 13: 19,
 14: 6,
 15: 21,
 16: 14,
 17: 9,
 18: 15,
 19: 10,
 20: 19}

Topic distributions per word

In [23]:
word_topic_counts['canada']

{1: 4,
 2: 4,
 3: 4,
 4: 2,
 5: 2,
 6: 2,
 7: 4,
 8: 0,
 9: 2,
 10: 1,
 11: 1,
 12: 0,
 13: 3,
 14: 1,
 15: 2,
 16: 1,
 17: 1,
 18: 2,
 19: 2,
 20: 2}

Total count of all topics

In [24]:
total_topic_counts

{1: 3717,
 2: 3732,
 3: 3742,
 4: 3732,
 5: 3727,
 6: 3584,
 7: 3716,
 8: 3776,
 9: 3635,
 10: 3616,
 11: 3669,
 12: 3787,
 13: 3723,
 14: 3753,
 15: 3752,
 16: 3678,
 17: 3590,
 18: 3691,
 19: 3676,
 20: 3722}

## Algorithm

### Hyperparameters

In [113]:
niter = 10  # Iterations of Gibbs sampler
alpha = 1  # Controls topic distribution per document
beta = 1  # Controls word distribution per topic

In [114]:
for _ in range(niter):
    for doc, words in title_docs_stemmed.items():
        for i, word in enumerate(words):
            #Z = 0
            densities = np.zeros(K)
            curr_topic = document_word_topics[doc][i]
            for k in range(1, K + 1):
                N_kj = document_topic_counts[doc].get(k, 0)
                N_wk = word_topic_counts[word].get(k, 0)
                N_k = total_topic_counts.get(k, 0)
                
                # New draw is conditioned on everything BUT this observation
                if curr_topic == k:
                    N_kj -= 1
                    N_wk -= 1
                    N_k -= 1
                    
                # Eq. 1
                a_kj = N_kj + alpha    
                b_wk = (N_wk + beta) / (N_k + W * beta)
                
                densities[k - 1] = a_kj * b_wk
                #Z += a_kj * b_wk
                
            # Draw a new topic
            densities /= np.sum(densities)  # Normalize
            new_topic = choices(range(1, K + 1), densities)[0]
            
            if new_topic == curr_topic:
                continue
            
            # Update counts
            document_word_topics[doc][i] = new_topic
            
            document_topic_counts[doc][curr_topic] -= 1
            document_topic_counts[doc][new_topic] += 1
            
            word_topic_counts[word][curr_topic] -= 1
            word_topic_counts[word][new_topic] += 1
            
            total_topic_counts[curr_topic] -= 1
            total_topic_counts[new_topic] += 1

## Estimating $\phi$ and $\theta$

Now that we have an estimate of the marginal posterior distribution of topics, we can get an estimate of:

$\phi_{wk}$: probability of word $w$ given $k$ 

and 

$\theta_{kj}$: mixture component of topic $k$ in document $j$

In [115]:
word_topic_counts['canada']

{1: 0,
 2: 0,
 3: 1,
 4: 5,
 5: 8,
 6: 0,
 7: 0,
 8: 1,
 9: 3,
 10: 0,
 11: 2,
 12: 0,
 13: 0,
 14: 0,
 15: 0,
 16: 0,
 17: 0,
 18: 17,
 19: 3,
 20: 0}

In [116]:
phi_matrix = np.zeros((K, W))

for w, word in enumerate(unique_words):
    for k in range(1, K + 1):
        N_wk = word_topic_counts[word][k]
        N_k = total_topic_counts[k]
        
        phi_matrix[k - 1, w] = (N_wk + beta) / (N_k + W * beta)

In [117]:
topic_top_words = {}

for k in range(K):
    phi_k = phi_matrix[k, :]
    top_10_idx = np.argsort(phi_k)[::-1][:10]
    top_10_words = [unique_words[i] for i in top_10_idx]
    topic_top_words[k + 1] = top_10_words

In [118]:
topic_top_words

{1: ['bank',
  'billion',
  'loan',
  'area',
  'feder',
  'state',
  'dlr',
  'credit',
  'program',
  'system'],
 2: ['equip',
  'l',
  'quot',
  'bern',
  'keidanren',
  'previous',
  'reiter',
  'v',
  'gener',
  'establish'],
 3: ['drug',
  'unit',
  'ici',
  'tax',
  'polit',
  'parti',
  'compani',
  'pharmaceut',
  'chemlawn',
  'call'],
 4: ['mln',
  'dlr',
  'vs',
  'ct',
  'net',
  'shr',
  'loss',
  'oper',
  'share',
  'profit'],
 5: ['would',
  'compani',
  'one',
  'presid',
  'week',
  'new',
  'two',
  'analyst',
  'secur',
  'market'],
 6: ['ton',
  'stock',
  'short',
  'compani',
  'januari',
  'copper',
  'system',
  'metal',
  'combin',
  'increas'],
 7: ['price',
  'dlr',
  'share',
  'offer',
  'oil',
  'quota',
  'produc',
  'compani',
  'agreement',
  'inc'],
 8: ['subsidiari',
  'servic',
  'inc',
  'j',
  'r',
  'requir',
  'n',
  'tobacco',
  'reynold',
  'chicago'],
 9: ['st',
  'western',
  'pacif',
  'inc',
  'motor',
  'pct',
  'canadian',
  'senior',
 

In [80]:
phi_wk = phi_matrix[0, :]

In [84]:
top_10 = np.argsort(phi_wk)[::-1][:10]

In [95]:
top_10_words = [unique_words[i] for i in top_10]

In [96]:
top_10_words

['said',
 'price',
 'countri',
 'opec',
 'govern',
 'say',
 'tonn',
 'meet',
 'last',
 'problem']

## Interpreting results

In [87]:
empir_word_topics = {word: max(counts, key=counts.get) for word, counts in word_topic_counts.items()}
#max(word_topic_counts['program'], key=word_topic_counts['program'].get)

In [88]:
empir_word_topics

{'535e4': 10,
 'product': 10,
 'therebi': 14,
 'simul': 20,
 'inexpens': 10,
 'chrysler': 10,
 'multin': 10,
 'hectic': 10,
 'bread': 11,
 'circul': 10,
 'slack': 20,
 'disappoint': 8,
 'amr': 10,
 'program': 10,
 'accordingli': 10,
 'recur': 12,
 'beach': 20,
 'inventori': 10,
 'tariff': 20,
 'humanist': 12,
 'dgsg': 20,
 'gm': 10,
 'ross': 1,
 'debat': 16,
 'conn': 9,
 'sight': 10,
 'hugh': 20,
 'wallaceburg': 10,
 'retriev': 10,
 'alan': 10,
 'wage': 20,
 'bag': 20,
 'jacqu': 1,
 'publicli': 10,
 'symbol': 2,
 'monopoli': 10,
 'bbc': 11,
 'nymex': 10,
 'modernis': 10,
 'dissatisfact': 10,
 'bureaucraci': 20,
 'amc': 10,
 'avail': 10,
 'acquir': 10,
 'inspector': 10,
 'innov': 15,
 'stoneciph': 10,
 'marcoss': 1,
 'evalu': 13,
 'meanwhil': 3,
 'copra': 20,
 'bumper': 2,
 'incept': 10,
 'wang': 20,
 'gun': 1,
 'though': 10,
 'express': 20,
 'materi': 20,
 'distanc': 1,
 'forum': 20,
 'infrastructur': 20,
 'llx': 16,
 'vendor': 20,
 'carniv': 1,
 'authorit': 2,
 'comptrol': 8,
 'arsarc